In [7]:
from time import ctime, time

import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
import numpy as np
from numba import njit, prange
from scipy.sparse.csgraph import laplacian
import networkx as nx

from graph_optimization import fdla_weights_symmetric, fmmc_weights, lmsc_weights, fastest_averaging_constant_weight, max_degree_weights, metropolis_hastings_weights
from lmsc import generateP

In [2]:
# adjacency matrices
As = [
	# all-to-all
	np.array([
		[0, 1, 1, 1, 1],
		[1, 0, 1, 1, 1],
		[1, 1, 0, 1, 1],
		[1, 1, 1, 0, 1],
		[1, 1, 1, 1, 0],
	]),
	# house
	np.array([
		[0, 1, 1, 0, 0],
		[1, 0, 1, 1, 0],
		[1, 1, 0, 0, 1],
		[0, 1, 0, 0, 1],
		[0, 0, 1, 1, 0],
	]),
	# ring
	np.array([
		[0, 1, 0, 0, 1],
		[1, 0, 1, 0, 0],
		[0, 1, 0, 1, 0],
		[0, 0, 1, 0, 1],
		[1, 0, 0, 1, 0],
	]),
	# line
	np.array([
		[0, 1, 0, 0, 0],
		[1, 0, 1, 0, 0],
		[0, 1, 0, 1, 0],
		[0, 0, 1, 0, 1],
		[0, 0, 0, 1, 0],
	]),
	# star
	np.array([
		[0, 1, 1, 1, 1],
		[1, 0, 0, 0, 0],
		[1, 0, 0, 0, 0],
		[1, 0, 0, 0, 0],
		[1, 0, 0, 0, 0],
	]),
	# 8-agents
	np.array([
		[0, 1, 1, 1, 0, 0, 0, 0],
		[1, 0, 1, 0, 1, 0, 0, 0],
		[1, 1, 0, 1, 1, 0, 0, 0],
		[1, 0, 1, 0, 1, 1, 1, 1],
		[0, 1, 1, 1, 0, 1, 1, 1],
		[0, 0, 0, 1, 1, 0, 1, 1],
		[0, 0, 0, 1, 1, 1, 0, 1],
		[0, 0, 0, 1, 1, 1, 1, 0],
	]),
]

# corresponding incidence matrices
Is = [
	# all-to-all
	np.array([
		[ 1,  1,  1,  1,  0,  0,  0,  0,  0,  0],
		[-1,  0,  0,  0,  1,  1,  1,  0,  0,  0],
		[ 0, -1,  0,  0, -1,  0,  0,  1,  1,  0],
		[ 0,  0, -1,  0,  0, -1,  0, -1,  0,  1],
		[ 0,  0,  0, -1,  0,  0, -1,  0, -1, -1],
	]),
	# house
	np.array([
		[ 1,  0,  0,  0, -1,  0],
		[ 0,  0,  0, -1,  1,  1],
		[-1,  1,  0,  0,  0, -1],
		[ 0,  0, -1,  1,  0,  0],
		[ 0, -1,  1,  0,  0,  0],
	]),
	# ring
	np.array([
		[ 1,  0,  0,  0, -1],
		[-1,  1,  0,  0,  0],
		[ 0, -1,  1,  0,  0],
		[ 0,  0, -1,  1,  0],
		[ 0,  0,  0, -1,  1],
	]),
	# line
	np.array([
		[ 1,  0,  0,  0],
		[-1,  1,  0,  0],
		[ 0, -1,  1,  0],
		[ 0,  0, -1,  1],
		[ 0,  0,  0, -1],
	]),
	# star
	np.array([
		[ 1,  1,  1,  1],
		[-1,  0,  0,  0],
		[ 0, -1,  0,  0],
		[ 0,  0, -1,  0],
		[ 0,  0,  0, -1],
	]),
	# 8-agents
	np.array([
		[  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
		[ -1,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
		[  0, -1,  0, -1,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
		[  0,  0, -1,  0,  0, -1,  0,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0],
		[  0,  0,  0,  0, -1,  0, -1, -1,  0,  0,  0,  1,  1,  1,  0,  0,  0],
		[  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0, -1,  0,  0,  1,  1,  0],
		[  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0, -1,  0, -1,  0,  1],
		[  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0, -1,  0, -1, -1],
	]),
]

# corresponding labels
labels = [
	'All-to-All',
	'House',
	'Ring',
	'Line',
	'Star',
	'8-Agents',
]

In [17]:
np.set_printoptions(threshold=99999999999999999, linewidth=99999999999999999)

for idx, (A, I) in enumerate(zip(As, Is)):
	with open(f'data/txt/{labels[idx]}.txt', 'w') as f:
		# f.write(f'P matrices for {labels[idx]}:\n\n')

		for kappa in [0.02, 0.3, 0.9]:
			P, _ = generateP(A, kappa)
			f.write(f'kappa = {kappa}:\n')
			f.write(f'```txt\n{np.round(P, 3)}\n```\n\n')

		# constant edge
		alpha, _, P, rho = fastest_averaging_constant_weight(I)
		f.write(f'constant-edge:\n')
		f.write(f'```txt\n{np.round(P, 3)}\n```\n\n')

		# maximum degree
		alpha, _, P, rho = max_degree_weights(I)
		f.write(f'max-degree:\n')
		f.write(f'```txt\n{np.round(P, 3)}\n```\n\n')

		# local degree (MH)
		_, P, rho = metropolis_hastings_weights(I)
		f.write(f'local-degree:\n')
		f.write(f'```txt\n{np.round(P, 3)}\n```\n\n')

		# fmmc
		_, P, rho = fmmc_weights(I)
		f.write(f'fmmc:\n')
		f.write(f'```txt\n{np.round(P, 3)}\n```\n\n')

		# fdla
		_, P, rho = fdla_weights_symmetric(I)
		f.write(f'fdla:\n')
		f.write(f'```txt\n{np.round(P, 3)}\n```\n\n')

		# lmsc
		_, P = lmsc_weights(I)
		f.write(f'lmsc:\n')
		f.write(f'```txt\n{np.round(P, 3)}\n```\n\n')